In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [4]:
## Loading the dataset
df = pd.read_csv('/Users/mohdsafeenkhan/Desktop/Machine_Learning/NLP/NLP_Project/Churn_modelling/Data/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Dropping irreleavant data column
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## Encode the categorical data
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
## One hot encoding Geography column
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo_encoder = ohe.fit_transform(df[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encode_df = pd.DataFrame(geo_encoder , columns=ohe.get_feature_names_out(['Geography']))
df = pd.concat([df.drop('Geography',axis=1),geo_encode_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [26]:
with open('label_encoder_gender.pkl','wb') as file :
    pickle.dump(label_encoder,file)
with open('ohe_encoder_geo.pkl','wb') as file:
    pickle.dump(ohe,file)

In [12]:
x = df.drop(columns=['Exited'],axis=1)
y = df['Exited']

In [ ]:
x

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.2,random_state=42)

In [15]:
# Applying Standard scaler
standardScaler = StandardScaler()
x_train = standardScaler.fit_transform(x_train)
x_test = standardScaler.transform(x_test)

In [27]:
with open('standardScaler.pkl','wb') as file:
    pickle.dump(standardScaler,file)

In [16]:
#Creating the neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [17]:
model = Sequential([
    Input(shape=(x_train.shape[1],)),# Input layer , provideing an input shape to our neural network
    Dense(units=64,activation='relu'),
    Dense(units=32,activation='relu'),
    Dense(units=1,activation='sigmoid')
])

In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
#compiling our model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
#setting up Tensorboard to capture logs
log_dirctory = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%hH%M%S")

tensorflow_callback = TensorBoard(log_dir=log_dirctory,histogram_freq=1)

In [21]:
#setting up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=15,restore_best_weights= True) # monitor is that which loss we need to monitor , patience is that this will wait for 5 epochs before stopping 

In [22]:
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100 ,callbacks=[tensorflow_callback ,early_stopping_callback ])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 875us/step - accuracy: 0.7972 - loss: 0.4564 - val_accuracy: 0.8350 - val_loss: 0.3881
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.8418 - loss: 0.3827 - val_accuracy: 0.8565 - val_loss: 0.3569
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.8534 - loss: 0.3543 - val_accuracy: 0.8580 - val_loss: 0.3535
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - accuracy: 0.8593 - loss: 0.3457 - val_accuracy: 0.8580 - val_loss: 0.3420
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.8586 - loss: 0.3417 - val_accuracy: 0.8585 - val_loss: 0.3435
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - accuracy: 0.8625 - loss: 0.3381 - val_accuracy: 0.8600 - val_loss: 0.3421
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.8615 - loss: 0.3340 - val_accuracy: 0.8630 - val_loss: 0.3464
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.8606 - loss: 0

In [23]:
#saving the model
model.save("model.h5")

In [24]:
#launch Tensowboard
%load_ext tensorboard

In [25]:
%tensorboard --logdir logs/fit --host localhost --port 6006